In [1]:
%matplotlib inline
import pandas as pd

In [2]:
delitos_df = pd.read_csv('clean_final_delitos.csv')

## PROBAR UN MODELO SUPERVISADO

In [6]:
data = delitos_df.copy()
for i, column in enumerate(list([str(d) for d in data.dtypes])):
    if column == "object":
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].mode())
        data[data.columns[i]] = data[data.columns[i]].astype("category").cat.codes
    else:
        data[data.columns[i]] = data[data.columns[i]].fillna(data[data.columns[i]].median())

In [7]:
data.head(5)

,Año,Clave_Ent_x,Entidad,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero_x,Febrero_x,Marzo_x,...,Junio_y,Julio_y,Agosto_y,Septiembre_y,Octubre_y,Noviembre_y,Diciembre_y,TotalDelitos_y,Promedio por entidad federativa,Por encima del promedio
0,2015,1,0,4,18,19,7,3,0.0,2.0,...,45.410395,44.585459,44.456633,45.021046,46.474809,44.20727,41.676658,528.63648,76.698069,0
1,2015,1,0,4,18,19,6,1,1.0,0.0,...,45.410395,44.585459,44.456633,45.021046,46.474809,44.20727,41.676658,528.63648,76.698069,0
2,2015,1,0,4,18,19,8,0,0.0,2.0,...,45.410395,44.585459,44.456633,45.021046,46.474809,44.20727,41.676658,528.63648,76.698069,0
3,2015,1,0,4,18,19,26,2,0.0,0.0,...,45.410395,44.585459,44.456633,45.021046,46.474809,44.20727,41.676658,528.63648,76.698069,0
4,2015,1,0,4,18,18,7,0,0.0,0.0,...,45.410395,44.585459,44.456633,45.021046,46.474809,44.20727,41.676658,528.63648,76.698069,0


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [12]:
y = data.pop('Tipo de delito')
X = data.copy()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

In [15]:
clf = RandomForestRegressor(n_estimators = 1000, max_depth = 4 )

In [16]:
clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, n_estimators=1000)

In [17]:
#metrica del modelo
from sklearn.metrics import r2_score

In [19]:
r2_score(y_test, clf.predict(X_test))

0.9972248203203581

## PROBAR UN MODELO NO SUPERVISADO O DE CLUSTERING PARA TEXTO

In [ ]:
import gensim
from ntlk.corpus import stopwords
import numpy as np
np.random.seed(2021)

In [ ]:
def preprocess(text):
    result = []
    for token in text.split():
        if token not in stopwords.words('spanish') and len(token)>4:
            result.append(token)
    return result

In [ ]:
process_business = [preprocess(x) for x in list(delitos_df['Tipo de delito'].unique())]

In [ ]:
dictionary = gensim.corpora.Dictionary(process_business)

In [ ]:
dictionary.filter_extremes(no_below=1, no_above=0.5)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in process_business]

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=6, id2word=dictionary, passes=10, workers =2 )

In [ ]:
for idx, type_bussiness in lda_model.print_topics(-1):
    print('Topic: {} Words:{}'.format(idx, type_bussiness))

In [ ]:
def get_max_topic(index):
    results = lda_model[bow_corpus[index]]
    return mx(lda_model[results], key = lambda x: x[1])[0]

In [ ]:
business_cluster = {x: get_max_topic(i) for i, x in enumerate(list(compras.df['Entidad'].unique()))}

In [ ]:
delitos_df['CLUSTER_NAME'] = delitos_df['Entidad'].apply(lambda x: business_cluster[x])

In [ ]:
delitos_df.value_counts()

In [ ]:
#promedio de los delitos por cluster
delitos_df.groupby('CLUSTER_NAME')['Enero'].mean().plot.bar()

In [ ]:
business_cluster